In [1]:
!pip install itables

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.7 MB/s eta 0:00:00


In [2]:
!pip install nltk

In [3]:
!pip install owlready2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 41.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for owlready2: filename=owlready2-0.46-cp310-cp310-linux_x86_64.whl size=24087766 sha256=3d50b26941c681fda8343f6b6bc92b55432b26ca0b9a32a9f5d1596211bf2659
  Stored in directory: /root/.cache/pip/wheels/bf/6f/ef/35df148baa59fc6b243aca4f101e17aed14b67dc1166b73fe0
Successfully built owlready2


In [4]:
#required libraries
from google.colab import files
from owlready2 import *
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import itables
from itables import show

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [5]:
#first uploading the abbreviated file and aircraft dataset to replace the abbreviation
from google.colab import files
def abbreviated_file (abb):
  abbreviations=pd.read(abb)
  return abbreviations

def aircraft(plane):
  aircraftData=pd.read(plane)
  return aircraftData

a=files.upload()
b=files.upload()


Saving Aviation_Abbreviation_Dataset.csv to Aviation_Abbreviation_Dataset.csv


Saving Aircraft_Annotation_DataFile.csv to Aircraft_Annotation_DataFile.csv


In [7]:
#replacing script
abbreviations_df = pd.read_csv("Aviation_Abbreviation_Dataset.csv")
abbreviation_mapping = dict(zip(abbreviations_df['Abbreviated'].str.lower(), abbreviations_df['Standard_Description'].str.upper()))

problems_actions_df = pd.read_csv("Aircraft_Annotation_DataFile.csv")

def replace_abbreviations(text):
    words = text.split()
    replaced_words = [abbreviation_mapping.get(word.lower(), word).upper() for word in words]
    return ' '.join(replaced_words)

# Apply the function to replace abbreviations in PROBLEM and ACTION columns
problems_actions_df['PROBLEM'] = problems_actions_df['PROBLEM'].apply(replace_abbreviations)
problems_actions_df['ACTION'] = problems_actions_df['ACTION'].apply(replace_abbreviations)

# Save the modified DataFrame to a new CSV file
problems_actions_df.to_csv("Aircraft_replaced_abbreviation.csv", index=False)

print("Replacement completed. Check 'Aircraft_replaced_abbreviation.csv' file.")

Replacement completed. Check 'Aircraft_replaced_abbreviation.csv' file.


In [6]:
#uploading the Aircraft_replaced_abbreviation to extract required terms

def read_dataset(csv_file):
  Aircraft=pd.read(csv_file)
  return Aircraft
dataset=files.upload()

Saving Aircraft_replaced_abbreviation.csv to Aircraft_replaced_abbreviation.csv


In [7]:

# Function to extract verbs and multi-word verbs using POS tagging
def extract_verbs(tagged_words):
    verbs = []
    i = 0
    while i < len(tagged_words):
        word, tag = tagged_words[i]
        if tag.startswith('VB') and len(word) > 2:
            verb_phrase = [word]
            j = i + 1
            # Include following verbs and particles in the verb phrase
            while j < len(tagged_words) and (tagged_words[j][1].startswith('VB') or tagged_words[j][1] in ['RP', 'RB']):
                verb_phrase.append(tagged_words[j][0])
                j += 1

            # Check if every word in the phrase is a verb or particle, if so, add to verbs list
            if all(tagged_word[1].startswith('VB') or tagged_word[1] in ['RP', 'RB'] for tagged_word in tagged_words[i:j]):
                verbs.append(' '.join(verb_phrase))

            i = j
        else:
            i += 1
    return verbs

df = pd.read_csv('Aircraft_replaced_abbreviation.csv')
df = df[['ACTION']].fillna('')

dft = df['ACTION'].str.lower().apply(word_tokenize)
dft2 = dft.apply(nltk.pos_tag)

df['verbs'] = dft2.apply(extract_verbs)

# Save the DataFrame with extracted verbs to a new CSV file
df.to_csv('verbs_extracted.csv', index=False)
show(df)

In [8]:
# unique verbs
all_verbs = []

# Iterate over the 'verbs' column and append verbs to the list
for verbs_list in df['verbs']:
    all_verbs.extend(verbs_list)

# Convert the list to a set to remove duplicates
unique_verbs = set(all_verbs)

# Print unique verbs
print(unique_verbs)
for verb in unique_verbs:
    print(verb)

{'missing', 'have previously been', 'tie', 'remains', 'found exhaust valve contacted', 'blended', 'finished reassembly', 'passed', 'review', 'take off', 'got', 'lowering', 'smiled near', 'replaced missing', 'drilled', 'ermoved', 'preserved', 'found r/h', 'galled', 'began disassembly', 'continued reassembly', 'completed transferring', 'remvoed', 'deterimed', 'lycoming', 'torn', 'wires', 'confirmed knocking', 'retorquing', 'repositioned', 'finished installing', 'done', 'attaching', 'fastened', 'run out', 'required', 'spe', 'fogged', 'decowled', 'aligned', 'broken', 'ran rough', 'considered', 'perform', 'assebled', 'prevent', 'baffle', 'refastened', 'replace', 'a/c up', 'starting', 'sealed lifting', 'sitting', 'overspeed found', 'were coming', 'tightened hardwared', 'replaced stand off', 'running rather', 'instaled', 'drain', 'cleaned tubes', 'did not notice', 'instructed', 'stuck', 'are', 'carburetor', 'check', 'filled', 'scroll', 'sealed up', 'dressed out', 'rmoved', 'disconnected', 'se

In [9]:
#remove uxiliarty verbs and some other ambigues words

unique_verbs = set(all_verbs)

# Create a set of auxiliary verbs
aux_verbs = set(['be', 'o/h', 'w/in','a/c','r/h', 'l/h', 'o/b','am', 'is', 'are', 'was', 'were', 'been', 'being', 'have', 'has', 'had', 'do', 'does', 'did', 'can', 'could', 'shall', 'should', 'will', 'would', 'may', 'might', 'must'])

# Remove auxiliary verbs that are alone
for verb in unique_verbs.copy():  # Use .copy() to avoid modifying a set while iterating over it
    if verb in aux_verbs and ' ' not in verb:
        unique_verbs.remove(verb)

# Print unique verbs without auxiliary verbs that are alone
print(unique_verbs)




{'missing', 'have previously been', 'tie', 'remains', 'found exhaust valve contacted', 'blended', 'finished reassembly', 'passed', 'review', 'take off', 'got', 'lowering', 'smiled near', 'replaced missing', 'drilled', 'ermoved', 'preserved', 'found r/h', 'galled', 'began disassembly', 'continued reassembly', 'completed transferring', 'remvoed', 'deterimed', 'lycoming', 'torn', 'wires', 'confirmed knocking', 'retorquing', 'repositioned', 'finished installing', 'done', 'attaching', 'fastened', 'run out', 'required', 'spe', 'fogged', 'decowled', 'aligned', 'broken', 'ran rough', 'considered', 'perform', 'assebled', 'prevent', 'baffle', 'refastened', 'replace', 'a/c up', 'starting', 'sealed lifting', 'sitting', 'overspeed found', 'were coming', 'tightened hardwared', 'replaced stand off', 'running rather', 'instaled', 'drain', 'cleaned tubes', 'did not notice', 'instructed', 'stuck', 'carburetor', 'check', 'filled', 'scroll', 'sealed up', 'dressed out', 'rmoved', 'disconnected', 'secured',

In [10]:
#now extracting location, part and problem
import re
import pandas as pd

# Load the DataFrame from the CSV file
df = pd.read_csv('Aircraft_replaced_abbreviation.csv')

# Fill missing values in the 'PROBLEM' column with an empty string
df['PROBLEM'].fillna('', inplace=True)

problem_pat = (
    '(?P<location>(?:(?:L|R)/H (?:ENG )?)?(?:CYL ?)?(?:#?\d(?: ?. \d)*)(?: CYL ?)?)? ?'
   '(?P<part>\w+(?:\s\w+)?)(?:\s[A-Z]+)?'
    '(?:\s(?:IS|ARE|HAS|HAVE)[,.]?)?'  # Ignore certain verbs with optional punctuation
    '(?:\sA)?'  # Ignore 'A' if present
    '\s(?P<problem>[A-Z\s,]+)'
)
# Extract problem information using the defined pattern
problem_extractions = df['PROBLEM'].str.extract(problem_pat, flags=re.IGNORECASE)

# Rename the columns for better understanding
problem_extractions.columns = ['Location', 'Part', 'Problem']

# Drop the 'ACTION' column
df.drop(columns=['ACTION'], inplace=True)

# Concatenate the extracted information DataFrame with the original DataFrame
result = pd.concat([df, problem_extractions], axis=1)

# Display the combined DataFrame
show(result)

**I have extracted all the required terms to be added to theontology and I have created theontology in the protege manually **bold text**

**In this following codes, I want to add annaotation *seeAlso* in the protege to subclasses of the part class, actually I found a website in whcih there are information about parts, so I have created a link to this page automatically using the following code.**

In [11]:
#uploading output file in which there are extracted location, parts, and problems
from google.colab import files
def a (file):
  b=pd.read(file)
  return b
s=files.upload()

Saving output.csv to output.csv


In [12]:
#generatin gthe URIs
import pandas as pd

# Load the DataFrame from the CSV file
df = pd.read_csv('output.csv')

# Define a function to generate the URL for each part name
def generate_part_url(part_name):
    base_url = 'https://www.flyaeroguard.com/learning-center/parts-of-an-airplane/'
    if isinstance(part_name, str):  # Check if part_name is a string
        # Replace spaces in part name with hyphens and convert to lowercase
        part_name_slug = part_name.replace(' ', '-').lower()
        return f'{base_url}#{part_name_slug}'
    else:
        return None

# Create a new column in the DataFrame containing the URLs
df['Webpage URL'] = df['Part'].apply(generate_part_url)

# Save the DataFrame with URLs to a new CSV file
show(df)

In [15]:
#uploading owl file
from google.colab import files
from owlready2 import *

def load_owl_file():
    uploaded = files.upload()
    file_name = next(iter(uploaded))
    # Load the ontology
    onto = get_ontology(file_name).load()

    return onto

# Call the function to upload and load the OWL file
ontology = load_owl_file()

Saving KRWproject.owl to KRWproject.owl


In [47]:
#now adding the seeAlso URIs to the ontology

# Load your ontology
onto = get_ontology("KRWproject.owl").load()

# Define the base URL for the webpage
base_url = "https://www.flyaeroguard.com/learning-center/parts-of-an-airplane/"

# Iterate over the subclasses of the Part class
for subclass in onto.Part.subclasses():
    # Create the URI for the webpage
    webpage_uri = base_url + "#" + subclass.name

    # Add the seeAlso annotation to the subclass
    subclass.seeAlso.append(webpage_uri)

# Save the modified ontology
onto.save("aircraft.owl")

**ok. now It is time to query the ontology**

In [17]:
!pip install rdflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.4 MB/s eta 0:00:00


In [48]:
#uploading owl file
from google.colab import files
from owlready2 import *

def load_owl_file():
    uploaded = files.upload()
    file_name = next(iter(uploaded))
    # Load the ontology
    onto = get_ontology(file_name).load()

    return onto

# Call the function to upload and load the OWL file
ontology = load_owl_file()

Saving aircraft.owl to aircraft (1).owl


**This SPARQL query retrieves all classes in the ontology along with their subclasses, if any. It uses the rdf:type predicate to find all instances of the OWL class (owl:Class). The rdfs:subClassOf predicate is used to link each class to its subclasses, and the OPTIONAL keyword ensures that even classes without subclasses are included in the result.**

In [50]:
from rdflib import Graph, URIRef
from rdflib.plugins.sparql import prepareQuery

g = Graph()
g.parse("aircraft.owl")

query = prepareQuery(
    """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT ?class ?subclass
    WHERE {
      ?class rdf:type owl:Class .
      OPTIONAL {
        ?subclass rdfs:subClassOf ?class .
      }
    }
    """
)

results = g.query(query)
for row in results:
    print(row)

(rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#Action'), rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#PepairProcedure'))
(rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#Problem'), rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#Broken'))
(rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#Problem'), rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#Crack'))
(rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#Problem'), rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#Damaged'))
(rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#Problem'), rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#Dirty'))
(rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#Problem'), rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#Leak'))
(rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#Problem'), rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#Loose'))
(rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#Problem'), rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#Shear'))
(rdfl

**This query retrieves the distinct classes (?event) from  ontology and counts the number of their subclasses (?subclass) using the COUNT aggregate function. The DISTINCT keyword ensures that only distinct results are returned.**

In [51]:
query = prepareQuery("""
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>

    SELECT DISTINCT ?event (COUNT(?subclass) AS ?subclassCount)
    WHERE {
      ?event rdf:type owl:Class .
      ?subclass rdfs:subClassOf ?event .
    }
    GROUP BY ?event
""")

for row in g.query(query):
    print(row)

(rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#Action'), rdflib.term.Literal('1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#Problem'), rdflib.term.Literal('10', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#Event'), rdflib.term.Literal('4', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#Part'), rdflib.term.Literal('8', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#Tool'), rdflib.term.Literal('1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#Component'), rdflib.term.Literal('2', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://Ameneh

**This query first calculates the count of subclasses for each class using a subquery and then finds the minimum count using the MIN keyword in the outer query.**

In [52]:

query = prepareQuery("""
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>

    SELECT ?event (MIN(?subclassCount) AS ?minSubclassCount)
    WHERE {
      {
        SELECT ?event (COUNT(?subclass) AS ?subclassCount)
        WHERE {
          ?event rdf:type owl:Class .
          ?subclass rdfs:subClassOf ?event .
        }
        GROUP BY ?event
      }
    }
    GROUP BY ?event
""")


for row in g.query(query):
    print(row)

(rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#Action'), rdflib.term.Literal('1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#Problem'), rdflib.term.Literal('10', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#Event'), rdflib.term.Literal('4', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#Part'), rdflib.term.Literal('8', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#Tool'), rdflib.term.Literal('1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://Ameneh.org/aircraft.owl#Component'), rdflib.term.Literal('2', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://Ameneh